In [ ]:
from Bio import Entrez

Entrez.email = 'me@byronjsmith.com'

chunksize = 50
total_available = float('inf')
num_returned = 0
idlist = []
while num_returned < total_available:
    print('requesting')
    result = Entrez.read(Entrez.esearch(db='assembly',
                               term=('txid2005473[Organism:exp] '
                                     'AND "latest genbank"[filter]) '
                                     'NOT "derived from surveillance project"[filter] '
                                     'NOT anomalous[filter]'),
                               retstart=num_returned, retmax=num_returned + chunksize))
    idlist.extend(result['IdList'])
    num_returned += int(result['RetMax'])
    total_available = int(result['Count'])
assert len(idlist) == total_available

In [ ]:
for assembly_id in idlist:
    _record = Entrez.read(Entrez.esummary(db='assembly', id=assembly_id, report='full'))
    record = _record['DocumentSummarySet']['DocumentSummary'][0]
    genbank_id = record['Synonym']['Genbank']
    species_name = record['SpeciesName']
    organism = record['Organism']
    species_taxid = record['SpeciesTaxid']
    isolate_name = record['Biosource']['Isolate']
    assembly_name = record['AssemblyName']
    print(genbank_id, assembly_name, species_name, organism, species_taxid, isolate_name, sep='\t')

In [ ]:
from glob import glob
from os.path import basename

for path in glob('/Users/byronsmith/Downloads/genome_assemblies_asm_stats/ncbi-genomes-2020-09-18/*_assembly_stats.txt'):
    stem = basename(path)[:-len('_assembly_stats.txt')]
    prefix_acc, suffix_acc, asmbl_name, *rest = stem.split('_', maxsplit=2)
    assert not rest
    with open(path) as f:
        isolate_lines = [l[len('# Isolate:  '):].strip() for l in f if l.startswith('# Isolate:')]
    print(f'{prefix_acc}_{suffix_acc}', asmbl_name, isolate_lines[0], sep='\t')